In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt
import xtrack as xt
import xcoll as xc
import xpart as xp
import xobjects as xo

# -10mm bump

In [2]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta2.json')

Loading line from dict:   0%|          | 0/36395 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
#Preping line for bump
env = line.env

line['bump.11207'] = 0
line['bump.11407'] = 0
line['bump.11607'] = 0
line['bump.12207'] = 0

line.ref['mdh.11207'].knl[0] += line.vars['bump.11207']
line.ref['mdh.11407'].knl[0] += line.vars['bump.11407']
line.ref['mdh.11607'].knl[0] += line.vars['bump.11607']
line.ref['mdh.12207'].knl[0] += line.vars['bump.12207']

In [8]:
#Context and parameters
context = xo.ContextCpu()

num_particles  = 1000
plane = 'DPneg'
sweep = 6000
sweep = -abs(sweep) if plane == 'DPpos' else abs(sweep)
num_turns = 6000

In [5]:
#Setting cavities to the same frequency and phase, plus setting the voltage
cavity_elements, cavity_names = line.get_elements_of_type(xt.Cavity)

for name in cavity_names:
    line[name].frequency = 200e6
    line[name].lag = 180
line['actcse.31632'].voltage = 3.5e6

In [6]:
#Installing TIDP
tidp_ap_tot = 147e-3
block_mvt = 29e-3

tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
line.collimators.install(names=['tidp.11434'], elements=[tidp])

Slicing line:   0%|          | 0/75252 [00:00<?, ?it/s]

In [7]:
tw = line.twiss()
opt = line.match(
    solve=False,
    start='mdh.11007',
    end='mdhw.11732',
    init=tw,
    vary=[
        xt.VaryList(['bump.11207', 'bump.11407', 'bump.11607'], step=1e-8, tag='bump',)
    ],
    targets = [
        xt.Target('x', -10e-3, at='tidp.11434'),
        xt.TargetSet(['x', 'px'], value=tw, at='mdhw.11732')
    ]
    )
opt.run_jacobian(10)

                                             
Optimize - start penalty: 0.09974                           
Matching: model call n. 11 penalty = 1.5340e-09              
Optimize - end penalty:  1.53398e-09                            


In [9]:
line.build_tracker()
part = xp.generate_matched_gaussian_bunch(nemitt_x=3.5e-6,
                                          nemitt_y=3.5e-6,
                                          sigma_z=0.224, num_particles=num_particles, line=line)

The line already has an associated tracker
*** Maximum RMS bunch length 0.2361011021355421m.
... distance to target bunch length: -2.2226e-01
... distance to target bunch length: 5.5784e-03
... distance to target bunch length: 5.2361e-03
... distance to target bunch length: -6.3267e-03
... distance to target bunch length: 1.9948e-03
... distance to target bunch length: -3.8258e-04
... distance to target bunch length: 5.3833e-05
... distance to target bunch length: 1.2594e-06
... distance to target bunch length: -1.0995e-10
... distance to target bunch length: 1.2805e-07
--> Bunch length: 0.22399999989004873
--> Emittance: 0.3170913627960028


In [10]:
line.discard_tracker()
line.build_tracker(_context=xo.ContextCpu(omp_num_threads='auto'))
rf_sweep = xc.RFSweep(line)
rf_sweep.info(sweep=sweep, num_turns=num_turns)

Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored
ld: warning: duplicate -rpath '/Users/lisepauwels/miniforge3/envs/xcoll/lib' ignored


Done compiling ContextCpu kernels.
The current frequency is 200000000.0Hz, adding 1.0Hz per turn until 200006000.0 (for 6000 turns).
This sweep will move the center of the bucket with Δδ = -0.01671417346401637 (-2.4723872573444057 buckets).


In [ ]:
line.scattering.enable()
rf_sweep.track(sweep=sweep, particles=part, num_turns=num_turns, time=True, with_progress=5)
line.scattering.disable()

Tracking:   0%|          | 0/6000 [00:00<?, ?it/s]